# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
#from pygeocoder import Geocoder
# Import API key
from key import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv(r"C:\Users\yenia\Desktop\JHU_Bootcamp\Homework\Whats-The-Weather-Like\WeatherPy\Data\cities.csv") 
print(len(weather_df))
weather_df.head()

572


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vaini,-21.20,-175.20,78.80,88,40,9.17,TO,1607284012
1,Kudahuvadhoo,2.67,72.89,82.51,78,100,14.29,MV,1607284012
2,Malanje,-9.54,16.34,62.65,95,80,1.57,AO,1607284012
3,Punta Arenas,-53.15,-70.92,68.00,45,75,26.40,CL,1607284012
4,Christchurch,-43.53,172.63,69.80,56,97,9.17,NZ,1607283830


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
#Drop NaN values
clean_weather_df = weather_df.dropna()
len(clean_weather_df)
#Store "Lat" and "Lng" into coordinates
coordinates = clean_weather_df[["Lat", "Lng"]].astype(float)
#store "Humidity"
humidity = clean_weather_df["Humidity"].astype(float)


fig = gmaps.figure()
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
under_80 = clean_weather_df.loc[clean_weather_df["Max Temp"] > 80,:]
ideal_temp = under_80.loc[under_80["Max Temp"] >70, :]
ideal_winds = ideal_temp.loc[ideal_temp["Wind Speed"] <10,:]
ideal_weather = ideal_winds.loc[ideal_winds["Cloudiness"] == 0,:]

ideal_weather_df = pd.DataFrame(ideal_weather)
ideal_weather_df
#print(len(ideal_weather_df))

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,Talca,-35.43,-71.67,89.60,24,0,4.70,CL,1607284016
297,Illapel,-31.63,-71.17,81.30,29,0,9.33,CL,1607284030
367,Kaffrine,14.11,-15.55,83.14,23,0,7.25,SN,1607284034
394,Terra Nova,-12.37,-40.67,83.88,44,0,4.47,BR,1607284036
477,Mount Isa,-20.73,139.50,80.60,26,0,8.05,AU,1607283810
502,Goundam,16.41,-3.67,80.60,23,0,8.32,ML,1607284042
504,Suleja,9.18,7.18,86.00,55,0,3.36,NG,1607284042


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
67,Talca,-35.43,-71.67,89.60,24,0,4.70,CL,1607284016,
297,Illapel,-31.63,-71.17,81.30,29,0,9.33,CL,1607284030,
367,Kaffrine,14.11,-15.55,83.14,23,0,7.25,SN,1607284034,
394,Terra Nova,-12.37,-40.67,83.88,44,0,4.47,BR,1607284036,
477,Mount Isa,-20.73,139.50,80.60,26,0,8.05,AU,1607283810,


In [7]:
hotel_df = ideal_weather_df
Hotel_Name = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
            "radius": 5000,
            "types" : "lodging",
            "key": g_key
}
for index, row in hotel_df.iterrows():
    try: 
        lat = row['Lat']
        lng = row['Lng']
        params['location'] = f"{lat}, {lng}"
        response = requests.get(base_url, params=params).json()
        results = response["results"]
        print({results[0]["name"]})
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
 
    except:
        print("No hotels found.")

        pass


{'Casa de Huespedes Marbargla'}
{'Hotel Londres'}
{'Beuleup counda motel'}
No hotels found.
{'ibis Styles Mt Isa Verona'}
No hotels found.
{'Capy Motel'}


In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
67,Talca,-35.43,-71.67,89.60,24,0,4.70,CL,1607284016,Casa de Huespedes Marbargla
297,Illapel,-31.63,-71.17,81.30,29,0,9.33,CL,1607284030,Hotel Londres
367,Kaffrine,14.11,-15.55,83.14,23,0,7.25,SN,1607284034,Beuleup counda motel
394,Terra Nova,-12.37,-40.67,83.88,44,0,4.47,BR,1607284036,
477,Mount Isa,-20.73,139.50,80.60,26,0,8.05,AU,1607283810,ibis Styles Mt Isa Verona
502,Goundam,16.41,-3.67,80.60,23,0,8.32,ML,1607284042,
504,Suleja,9.18,7.18,86.00,55,0,3.36,NG,1607284042,Capy Motel


In [9]:
hotel_df["Hotel Name"].replace("", np.nan, inplace = True)
hotel_df.dropna(subset = ["Hotel Name"], inplace = True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
67,Talca,-35.43,-71.67,89.60,24,0,4.70,CL,1607284016,Casa de Huespedes Marbargla
297,Illapel,-31.63,-71.17,81.30,29,0,9.33,CL,1607284030,Hotel Londres
367,Kaffrine,14.11,-15.55,83.14,23,0,7.25,SN,1607284034,Beuleup counda motel
477,Mount Isa,-20.73,139.50,80.60,26,0,8.05,AU,1607283810,ibis Styles Mt Isa Verona
504,Suleja,9.18,7.18,86.00,55,0,3.36,NG,1607284042,Capy Motel


In [10]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)
coordinates = list(zip(hotel_df.Lat,hotel_df.Lng))
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…